# Grid search CV

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading data

In [7]:
df = sns.load_dataset("iris")
df.head()
#We will be predicting species

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# train test split and dropping output values

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
X = df.drop("species", axis=1)
y = df["species"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# KNN 

In [15]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors = 5)

model_knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [17]:
model_knn.score(X_test, y_test)

0.98

# SVM

In [24]:
from sklearn.svm import SVC
model_svm = SVC(gamma = "auto")

model_svm.fit(X_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'auto'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [25]:
model_svm.score(X_test, y_test)

1.0

In [32]:
#Now lets use grid search CV
from sklearn.model_selection import GridSearchCV
classifier = GridSearchCV((model_svm), {  # these all values are being added using the docs
    "C": [1,10,20,30],
    "kernel": ["rbf", "linear"],
}, cv = 5, return_train_score = False)

In [34]:
classifier.fit(X, y)

,estimator,SVC(gamma='auto')
,param_grid,"{'C': [1, 10, ...], 'kernel': ['rbf', 'linear']}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,1


In [37]:
classifier.cv_results_

{'mean_fit_time': array([0.00283694, 0.00168009, 0.00139508, 0.0010736 , 0.00104361,
        0.00114703, 0.00109906, 0.0010076 ]),
 'std_fit_time': array([4.03995790e-04, 3.76111710e-04, 2.44519686e-04, 8.46177062e-05,
        3.15269786e-05, 2.39752993e-04, 4.40922015e-05, 2.99062443e-05]),
 'mean_score_time': array([0.00226536, 0.0014482 , 0.00118852, 0.00091109, 0.00082254,
        0.00082774, 0.00082722, 0.00077   ]),
 'std_score_time': array([6.46460845e-04, 4.01977752e-04, 3.62170394e-04, 1.63451376e-04,
        2.64337292e-05, 1.27075394e-04, 3.29109538e-05, 2.65005656e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=np.str_('?'),
        

In [40]:
#Here we have converted the result into the df so that we can see it properly
result = pd.DataFrame(classifier.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002837,0.000404,0.002265,0.000646,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001680,0.000376,0.001448,0.000402,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001395,0.000245,0.001189,0.000362,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001074,0.000085,0.000911,0.000163,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001044,0.000032,0.000823,0.000026,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001147,0.000240,0.000828,0.000127,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6
6,0.001099,0.000044,0.000827,0.000033,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.933333,1.0,0.960000,0.038873,7
7,0.001008,0.000030,0.000770,0.000027,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,7


In [42]:
#Here we have printed only those col which we needed
result[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667
6,30,rbf,0.960000
7,30,linear,0.960000


# Random search CV

In [48]:
#In random search CV, we will get random results of how many we have specified
from sklearn.model_selection import RandomizedSearchCV
classifier_r = RandomizedSearchCV((model_svm),{
    "C": [1,10,20,30],
    "kernel": ["rbf", "linear"],
}, n_iter=4, cv=5, return_train_score = False)

In [49]:
classifier_r.fit(X,y)

,estimator,SVC(gamma='auto')
,param_distributions,"{'C': [1, 10, ...], 'kernel': ['rbf', 'linear']}"
,n_iter,4
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [51]:
classifier_r.cv_results_

{'mean_fit_time': array([0.00317097, 0.00267711, 0.00260301, 0.00237613]),
 'std_fit_time': array([0.00037698, 0.00024591, 0.000181  , 0.00031078]),
 'mean_score_time': array([0.00236721, 0.00197539, 0.0016974 , 0.00209999]),
 'std_score_time': array([0.00028901, 0.0001021 , 0.00035099, 0.00071472]),
 'param_kernel': masked_array(data=['linear', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_C': masked_array(data=[30, 20, 1, 1],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'kernel': 'linear', 'C': 30},
  {'kernel': 'linear', 'C': 20},
  {'kernel': 'rbf', 'C': 1},
  {'kernel': 'linear', 'C': 1}],
 'split0_test_score': array([1.        , 1.        , 0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1., 1., 1.]),
 'split2_test_score': array([0.9       , 0.9       , 0.96666667, 0.96666667]),
 'split3_test_score': array([0.9       , 0.93333

In [57]:
results = pd.DataFrame(classifier_r.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003171,0.000377,0.002367,0.000289,linear,30,"{'kernel': 'linear', 'C': 30}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,4
1,0.002677,0.000246,0.001975,0.000102,linear,20,"{'kernel': 'linear', 'C': 20}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,3
2,0.002603,0.000181,0.001697,0.000351,rbf,1,"{'kernel': 'rbf', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002376,0.000311,0.002100,0.000715,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1


In [56]:
results[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,30,linear,0.960000
1,20,linear,0.966667
2,1,rbf,0.980000
3,1,linear,0.980000
